## This notebook is the main code for the triangulation method

we start with imports the required packages

In [1]:
#imports
import numpy as np
import glob
import os
from my_utils import yolo2bbox
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from math import radians, sin, degrees, atan
import json

set the paths to the frames and the labels, each sequence consistst of 30 frames

In [27]:
#set path to yolo labels
lab_path = r'C:\Users\portierl4527\ARCADIS\Afstuderen Levi - Depth estimation - General\yolov5\yolov5-master\runs\detect\low_quality\labels'
labels = glob.glob(lab_path+'\*.txt')

#set path to images
image_path = r'C:\Users\portierl4527\ARCADIS\Afstuderen Levi - Depth estimation - General\Cityscapes-Sequence\leftImg8bit_sequence_trainvaltest\low_quality'
images = glob.glob(image_path+'\*.png')

#get the names of frames
framenames = []

for file in images:
    
    basename = os.path.basename(file) #filename
    basename_no_ext = os.path.splitext(basename)[0]
    framenames.append(basename_no_ext)

#this is the first frame of each sequence
framenames[::30][:10]

['bonn_000000_000000_low',
 'bonn_000001_000000_low',
 'bonn_000002_000000_low',
 'bonn_000003_000000_low',
 'bonn_000004_000000_low',
 'bonn_000005_000000_low',
 'bonn_000006_000000_low',
 'bonn_000007_000000_low',
 'bonn_000008_000000_low',
 'bonn_000009_000000_low']

### Object tracking

* This is the main code used to track an object in multiple detections.
* We pass a detection of a single object in the first frame.
* For each detection we track the object in the next frames in the sequence.
* While doing this each x coordinated is saved.
* Once the end of the sequence is reached or the object moves out if reach the trackers rests and moves to the next object.
* We return for each objects a list of frames and x coordinates.
\
\
* There is an option to track multiple objects at once which is commented.

In [28]:
def template_matching(image_directory , framenames, index, bbox):
    
    
    img1 = cv2.imread(os.path.join(image_directory, framenames[index[0]]+'.png')) #read first frame in sequence
    #img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    
    
#     multitracker = cv2.legacy.MultiTracker_create()  
    
#     for bbox in bbox_list:
#         tracker = cv2.legacy.TrackerCSRT_create()
#         multitracker.add(tracker, img1, bbox)

    tracker = cv2.TrackerCSRT_create() #create tracker

    success = tracker.init(img1, bbox) #add image and bbox to tracker

    
    #add center of bbox to the list
    (x,y,w,h) = [int(v) for v in bbox]
    x_offset = [int(x-h/2+w/2)]
    y_offset = [int(y-h/2+w/2)]
    
    capture = [framenames[index[0]]] #add first frame to the list
     
    framecounter = 1 #keep track of frames
    
    while True:
        
        #read the next frame
        try:
            img2 = cv2.imread(os.path.join(image_directory, framenames[index[0]+framecounter]+'.png'))
            #img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
        
        except:
            print('No more frames in this sequence')
            break
            
#         if img2 is None:
#             break         
#         (success, bboxes) = multitracker.update(img2)

        #update tracker with the next frame
        success, bbox = tracker.update(img2)
        bbox = tuple([int(b) for b in bbox])
        

        #if object is tracked succesfully
        if success:    
            (x,y,w,h) = [int(v) for v in bbox]
            
            
            #if x is inside the image
          
            x_offset.append(int(x-h/2+w/2)) #append center of bbox
            capture.append(framenames[index[0]+framecounter]) #append name of next frame
           
        
        framecounter += 1 #go to the next frame
        
        if framecounter == 30: #make sure it doesn't move to the next section
            break
        
        #if tracker couldn't find object in next frame exit the loop
        if not success: 
            
            break
            

    return x_offset, capture

In [29]:
object_ids = []
list_x_offset = []
list_capture = []
coordinates=[]

#for each first frame of the sequence
for sequence in tqdm(framenames[::30]):
    index = [i for i, val in enumerate(framenames) if val in sequence] #index of new sequences in list of all frames
    
    if os.path.isfile(os.path.join(lab_path, sequence+'.txt')): #check if detection is available
        
        bbox_list = []
        
        #extract coordinates from the label
        label_norm = np.loadtxt(os.path.join(lab_path,sequence+'.txt')).reshape(-1, 5)
        for i in range(len(label_norm)):
            labels_conv=label_norm[i]
            object_id = labels_conv[0]
            
            xmin, xmax, ymin, ymax =  yolo2bbox(labels_conv[1], labels_conv[2], labels_conv[3], labels_conv[4])
            ROI = [xmin,ymin,xmax-xmin, ymax-ymin]
            
            #remove all street signs from the detection
            if object_id == 0 and (xmax-xmin > 2*(ymax-ymin) or ymax-ymin > 2* (xmax-xmin)):
                continue
            else:    
                bbox = tuple([int(x) for x in ROI])
                bbox_list.append(bbox)
                object_ids.append(object_id)
        
        #for detected object in the image
        for bbox in bbox_list:
            
            #run tracker
            x_offset, capture = template_matching(image_path, framenames, index, bbox)
            
            #append x coordinate, framname and coordinates to lis
            list_x_offset.append(x_offset)
            list_capture.append(capture)
            coordinates.append(bbox)
                    
    else: 
        continue


100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [02:57<00:00,  2.53s/it]


In [31]:
time_path =  r'C:\Users\portierl4527\ARCADIS\Afstuderen Levi - Depth estimation - General\Cityscapes-Sequence\timestamp_sequence\timestamp_sequence\test'

vehicle_path = r'C:\Users\portierl4527\ARCADIS\Afstuderen Levi - Depth estimation - General\Cityscapes-Sequence\vehicle_sequence\test'

timestamps = []
all_speeds = []
distances = []
###################################### data.frame
for sequence in tqdm(list_capture):
    
    timestamp =[]
    speeds = []
    
    for i in sequence:

        name = i[:-4]

        time_file = os.path.join(time_path,'{}_timestamp.txt'.format(name))
        vehicle_file = os.path.join(vehicle_path,'{}_vehicle.json'.format(name))
        
        with open(time_file, 'r') as file:
            lines=file.read()

            nanoseconds = int(lines.strip())
            seconds = nanoseconds/1000000000

        timestamp.append(seconds)
        
   
        with open(vehicle_file, 'r') as f:
            file_data = json.load(f)

        speed = file_data['speed']
   
        speeds.append(speed)
    
        dist = []
        i=0
        j=1

        while j < len(speeds):
            avg_speed= (speeds[i+1]+speeds[i])/2 #average speed
            time_diff = (timestamp[i+1] - timestamp[i]) #time difference
            distance = avg_speed*time_diff
            i+=1 #move to the next value in the column.
            j+=1 #next value in the new column.
            dist.append(distance)
        dist.insert(0,0)
    
    
    distances.append(dist)
    timestamps.append(timestamp)
    all_speeds.append(speeds)
    

100%|█████████████████████████████████████████████████████████████████████████████████| 99/99 [00:00<00:00, 163.45it/s]


In [32]:
data = {'object_id': object_ids,
        'x': list_x_offset,
       'frame': list_capture,
        'coordinates': coordinates,
       'distances': distances,
       'timestamps': timestamps}

df = pd.DataFrame(data)
df.head()

,object_id,x,frame,coordinates,distances,timestamps
0,1.0,"[243, 234, 231, 226, 216, 211, 205, 193, 185, ...","[bonn_000001_000000_low, bonn_000001_000001_lo...","(266, 295, 43, 88)","[0, 0.36129937538100004, 0.18407185902000003, ...","[0.0, 0.117630816, 0.176446176, 0.235261528, 0..."
1,1.0,"[284, 275, 271, 267, 258, 253, 248, 235, 231, ...","[bonn_000001_000000_low, bonn_000001_000001_lo...","(308, 294, 38, 86)","[0, 0.36129937538100004, 0.18407185902000003, ...","[0.0, 0.117630816, 0.176446176, 0.235261528, 0..."
2,0.0,"[1305, 1313, 1321, 1324, 1330, 1332, 1334, 133...","[bonn_000003_000000_low, bonn_000003_000001_lo...","(1314, 375, 28, 45)","[0, 1.1186106251320003, 1.118610320828, 0.5590...","[0.0, 0.117630856, 0.23526168, 0.294077072, 0...."
3,0.0,"[1325, 1334, 1342, 1344, 1349, 1351, 1351, 135...","[bonn_000003_000000_low, bonn_000003_000001_lo...","(1326, 206, 48, 50)","[0, 1.1186106251320003, 1.118610320828, 0.5590...","[0.0, 0.117630856, 0.23526168, 0.294077072, 0...."
4,0.0,"[1202, 1204, 1205, 1209, 1209, 1211, 1215, 121...","[bonn_000004_000000_low, bonn_000004_000001_lo...","(1204, 328, 23, 26)","[0, 1.1865671271510003, 0.5959734207454999, 0....","[0.0, 0.117630752, 0.176446216, 0.235261528, 0..."


In [33]:
GT_PATH = r'C:\Users\portierl4527\ARCADIS\Afstuderen Levi - Depth estimation - General\Cityscapes-Sequence\disparity_sequence_trainvaltest\gt_depth'
gt_labels = glob.glob(GT_PATH+'\*.npy')

y_true = []

for index, sequence in tqdm(enumerate(df['frame'])):
        
    name = sequence[0][:-4]
   
    gt_file = os.path.join(GT_PATH,'{}_disparity.npy'.format(name))

    object_gt = np.load(gt_file)
    coords = df.coordinates.loc[index]
   
    bbox = (coords[0], coords[2]+coords[0], coords[1], coords[3]+coords[1]) #xmin, xmax, ymin, ymax
    
    true_distance = np.median(object_gt[bbox[2]:bbox[3],bbox[0]:bbox[1]]) #add the true distance
    y_true.append(true_distance)

df['y_true']=y_true   
df['y_true']=df['y_true'].round(2)

index_names = df.loc[df['y_true'] > 60 ].index
  
# drop these row indexes
# from dataFrame
df.drop(index_names, inplace = True)
df.reset_index()
df.head()


99it [00:00, 276.47it/s]


,object_id,x,frame,coordinates,distances,timestamps,y_true
0,1.0,"[243, 234, 231, 226, 216, 211, 205, 193, 185, ...","[bonn_000001_000000_low, bonn_000001_000001_lo...","(266, 295, 43, 88)","[0, 0.36129937538100004, 0.18407185902000003, ...","[0.0, 0.117630816, 0.176446176, 0.235261528, 0...",22.90
1,1.0,"[284, 275, 271, 267, 258, 253, 248, 235, 231, ...","[bonn_000001_000000_low, bonn_000001_000001_lo...","(308, 294, 38, 86)","[0, 0.36129937538100004, 0.18407185902000003, ...","[0.0, 0.117630816, 0.176446176, 0.235261528, 0...",22.82
2,0.0,"[1305, 1313, 1321, 1324, 1330, 1332, 1334, 133...","[bonn_000003_000000_low, bonn_000003_000001_lo...","(1314, 375, 28, 45)","[0, 1.1186106251320003, 1.118610320828, 0.5590...","[0.0, 0.117630856, 0.23526168, 0.294077072, 0....",40.22
3,0.0,"[1325, 1334, 1342, 1344, 1349, 1351, 1351, 135...","[bonn_000003_000000_low, bonn_000003_000001_lo...","(1326, 206, 48, 50)","[0, 1.1186106251320003, 1.118610320828, 0.5590...","[0.0, 0.117630856, 0.23526168, 0.294077072, 0....",42.60
5,0.0,"[1857, 1857, 1884, 1916, 1944, 1944, 1936, 195...","[bonn_000004_000000_low, bonn_000004_000001_lo...","(1844, 215, 65, 38)","[0, 1.1865671271510003, 0.5959734207454999, 0....","[0.0, 0.117630752, 0.176446216, 0.235261528, 0...",13.46


In [34]:
cam_path = r'C:\Users\portierl4527\ARCADIS\Afstuderen Levi - Depth estimation - General\Cityscapes-Dataset\00_Source_Data\camera\bonn_camera.json'
vehicle_path = r'C:\Users\portierl4527\ARCADIS\Afstuderen Levi - Depth estimation - General\Cityscapes-Sequence\vehicle_sequence\test'

def get_params(cam_path):
    
    with open(cam_path,'r') as f:
        file_data = json.load(f)
    yaw = file_data['extrinsic']['yaw']  
    fx = file_data['intrinsic']['fx']
    u0 = file_data['intrinsic']['u0']
    
    return yaw, fx, u0

def get_vehicle_bearing(framenames, timestamps):
    
    yaw_rates = []
    for i in range(len(framenames)):
        vehicle_file = os.path.join(vehicle_path,'{}_vehicle.json'.format(framenames[i][:-4]))
        

        with open(vehicle_file, 'r') as f:
                file_data = json.load(f)

        yaw_rate = file_data['yawRate']
        
        yaw_rates.append(yaw_rate)
        
        bearings = []
        idx=0
        j=1

        while j < len(yaw_rates):
            time_diff = (timestamp[idx+1] - timestamp[idx]) #time difference
            bearing = yaw_rates[j]*time_diff
            idx+=1 #move to the next value in the column.
            j+=1 #next value in the new column.
            bearings.append(bearing)
        bearings.insert(0, yaw_rates[0])
        
                   

    start_bearing = bearings[0]
    end_bearing = sum(bearings)

                       
    return start_bearing, end_bearing


def triangulate(offsets, distance, cam_path, framenames, timestamps):
    
    #camera paths
    if framenames[0].startswith('bonn'):
        cam_path = r'C:\Users\portierl4527\ARCADIS\Afstuderen Levi - Depth estimation - General\Cityscapes-Dataset\00_Source_Data\camera\bonn_camera.json'
    else: 
        cam_path = r'C:\Users\portierl4527\ARCADIS\Afstuderen Levi - Depth estimation - General\Cityscapes-Dataset\00_Source_Data\camera\mainz_camera.json'
    
    yaw,fx,u0  =get_params(cam_path)
    
    #offset
    x1 = abs(offsets[0]-u0)
 
    #do not return negative values
#     index = 1
#     while abs(offsets[-index]-u0) < 0:
#         index += 1
    
    x2 = abs(offsets[-1]-u0)
  
    dist = sum(distance[:])
    
    start_bearing, end_bearing  = get_vehicle_bearing(framenames, timestamps)
    A = degrees(atan((x1)/fx)) + yaw + degrees(start_bearing)
    
    B = 180 - degrees(atan((x2)/fx)) + yaw + degrees(end_bearing)

    C = 180 - A - B

    #math.sin(radians(C))/distance = sin(radians(B))/object_distance

    #object_distance*sin(radians(C)) = distance*sin(radians(B))

    object_distance = dist*sin(radians(B))/sin(radians(C))

    return object_distance

In [35]:
df = df.reset_index()
#df = df.drop('level_0', axis=1)
df.head()

,index,object_id,x,frame,coordinates,distances,timestamps,y_true
0,0,1.0,"[243, 234, 231, 226, 216, 211, 205, 193, 185, ...","[bonn_000001_000000_low, bonn_000001_000001_lo...","(266, 295, 43, 88)","[0, 0.36129937538100004, 0.18407185902000003, ...","[0.0, 0.117630816, 0.176446176, 0.235261528, 0...",22.90
1,1,1.0,"[284, 275, 271, 267, 258, 253, 248, 235, 231, ...","[bonn_000001_000000_low, bonn_000001_000001_lo...","(308, 294, 38, 86)","[0, 0.36129937538100004, 0.18407185902000003, ...","[0.0, 0.117630816, 0.176446176, 0.235261528, 0...",22.82
2,2,0.0,"[1305, 1313, 1321, 1324, 1330, 1332, 1334, 133...","[bonn_000003_000000_low, bonn_000003_000001_lo...","(1314, 375, 28, 45)","[0, 1.1186106251320003, 1.118610320828, 0.5590...","[0.0, 0.117630856, 0.23526168, 0.294077072, 0....",40.22
3,3,0.0,"[1325, 1334, 1342, 1344, 1349, 1351, 1351, 135...","[bonn_000003_000000_low, bonn_000003_000001_lo...","(1326, 206, 48, 50)","[0, 1.1186106251320003, 1.118610320828, 0.5590...","[0.0, 0.117630856, 0.23526168, 0.294077072, 0....",42.60
4,5,0.0,"[1857, 1857, 1884, 1916, 1944, 1944, 1936, 195...","[bonn_000004_000000_low, bonn_000004_000001_lo...","(1844, 215, 65, 38)","[0, 1.1865671271510003, 0.5959734207454999, 0....","[0.0, 0.117630752, 0.176446216, 0.235261528, 0...",13.46


In [36]:
y_pred = []
for i in range(len(df)):

    prediction = triangulate(df.x[i], df.distances[i], cam_path, df.frame[i], df.timestamps[i])
    y_pred.append(prediction)
    
df['y_pred'] = y_pred
df['y_pred'] = df['y_pred'].round(2)

In [37]:
from sklearn.metrics import mean_absolute_error as MAE
df.loc[(df.y_pred < 0), 'y_pred'] = 0
#df.loc[(df.y_pred > 60), 'y_pred'] = 60



print(MAE(df.y_true, df.y_pred))

18.44923076923077


In [38]:
df.to_pickle('triangulation_low_results.pkl')